In [2]:
# opencv 설치

# !pip install opencv-python

In [4]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras as tf_keras

cv2.__version__

'4.10.0'

In [5]:
# 모델 가져오기
model = tf_keras.models.load_model("models/mnist-cnn-model.keras")


In [29]:
cap = cv2.VideoCapture(0) # 카메라 열기

if cap.isOpened():
    # print("camera is opened")    
    while True:
        ret, img = cap.read()
        if ret:
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # 흑백이미지로 변환
            # threshold, bin_img = cv2.threshold(gray_img, 110, 255, cv2.THRESH_BINARY)     # 이미지 이진화 (흑백으로 대비)
            threshold, bin_img = cv2.threshold(gray_img, 110, 255, cv2.THRESH_BINARY_INV)   # 이미지 이진화 (흑백으로 대비)

            contours, hierachy = \
                cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # 경계영역(경계선) 찾기
            
            for contour in contours:
                (x, y), radius = \
                    cv2.minEnclosingCircle(contour) # 경계영역을 포함하는 가장 작은 원 찾기
                if radius > 3:
                    # 원으로부터 사각형 정보 도출
                    xs, xe = int(x-radius), int(x+radius)
                    ys, ye = int(y-radius), int(y+radius)
                    # 이미지에 사각형 그리기
                    cv2.rectangle(bin_img, (xs, ys), (xe, ye), (200, 0, 0), 1)

                    roi = bin_img[ys:ye, xs:xe]
                    if roi.shape[0] == 0 or roi.shape[1] == 0:
                        continue
                    dst = cv2.resize(roi, dsize=(50, 50), interpolation=cv2.INTER_AREA)
                    dst = cv2.resize(dst, dsize=(26, 26), interpolation=cv2.INTER_AREA)
                    input = np.zeros((28, 28)) # 모델에 입력할 이미지
                    input[1:-1, 1:-1] = dst[:, :]
                    input = input.reshape((-1, 28, 28, 1))

                    output = model.predict(input, verbose=0)   # 예측모델에 이미지를 통과시켜서 예측 수행
                    digit = output.argmax()         # 예측 결과에서 CLASS 값 찾기 
                    cv2.putText(bin_img, str(digit), (xs, ys), cv2.FONT_ITALIC, 2, (200, 0, 0))

            cv2.imshow("Image", bin_img) # 창 생성하고 창에 이미지 표시
            if cv2.waitKey(1)&0xFF == 27: # ESC Key가 눌린 경우
                break
else:
    print("fail to open camera")

cap.release() # 카메라 닫기
cap.release()
cap.release()
cap.release()
cap.release()
cv2.destroyAllWindows()